In [ ]:
from tkinter import *
import csv 
import datetime
import matplotlib.pyplot as plt
import operator

f = open('local_data_cp949.csv', encoding='cp949')
data = csv.reader(f)
next(data)
data = list(data)

#-----------------------------------------------------
# 하위목표 1: 2018년 제주도 월별 이용금액 추이 확인
#-----------------------------------------------------
def analysis1():
    sum_mon = [0]*12
    for row in data:
        if row[8][:4] == '2018':                        # 2018년 데이터만 대상으로
            month = int(row[8][4:]) - 1                 # row의 월 확인
            sum_mon[month]+=int(row[-1])                # row의 달에 해당하는 위치에 이용금액 합산
    for i, sum in enumerate(sum_mon):
        sum_mon[i] = sum/10000000000                    # 단위 수정

    # 그래프 그리기
    plt.figure(figsize=(8, 4))
    plt.rc('font', family='Malgun Gothic')
    plt.title('제주도 내 월별 카드 이용금액 그래프')
    plt.bar(range(1, 13), sum_mon, color='orange')
    plt.xlabel('월별')
    plt.ylabel('이용금액 (단위:100억)')
    plt.xticks(range(0,13))
    plt.ylim([10,15])
    plt.plot(range(1, 13), sum_mon, color='green')
    plt.savefig('graph.png')
    plt.show()
    
    graph_img = PhotoImage(file='graph.png')            # tkinter 화면에 노출
    graph_lbl.configure(image=graph_img)
    graph_lbl.image = graph_img

#----------------------------------------------------------------------------    
# 하위목표 2: 가장 높은 카드 이용금액을 보인 달의 업종별 카드 이용 비율 확인
#----------------------------------------------------------------------------
def analysis2():    
    max_sum_mon = max_mon()                            # 높은 이용금액 달 확인
    
    sum_shop = {}

    for row in data:
        if row[8][4:] == max_sum_mon:                  # 해당 월에 해당하는 데이터만 업종별 합산
            shop = row[4]
            if shop not in sum_shop.keys():
                sum_shop[shop] = int(row[-1])
            else:
                sum_shop[shop] += int(row[-1])
   
    # 그래프 그리기
    plt.figure(figsize=(8, 4))
    plt.title(max_sum_mon+"월 업종별 카드 이용금액 그래프")
    sum_shop = dict(sorted(sum_shop.items(), key=operator.itemgetter(1), reverse = True))
    plt.pie(sum_shop.values(), labels = sum_shop.keys(), autopct='%1.1f%%')
    plt.savefig('graph.png')
    plt.show()
    
    graph_img = PhotoImage(file='graph.png')
    graph_lbl.configure(image=graph_img)
    graph_lbl.image = graph_img
    
#-----------------------------------------------------    
# 하위목표 3: 남녀 업종별 카드 이용금액 비교
#-----------------------------------------------------
def analysis3():
    # 빈 딕셔너리 생성
    male =  {'교통': [0]*12, '기타': [0]*12, '소매': [0]*12, '쇼핑': [0]*12, '숙박': [0]*12, '유흥': [0]*12, '문화/레져': [0]*12, '식음료': [0]*12}
    female = {'교통': [0]*12, '기타': [0]*12, '소매': [0]*12, '쇼핑': [0]*12, '숙박': [0]*12, '유흥': [0]*12, '문화/레져': [0]*12, '식음료': [0]*12}

    for row in data:
        shop=row[4]
        month = int(row[-4][4:])-1
        if row[7] == '여':
            female[shop][month]+=int(row[-1])               # 여성 데이터 합산
        elif row[7] == '남':
            male[shop][month]+=int(row[-1])                 # 남성 데이터 합산

    male_sum = {}
    female_sum = {}

    for shop in female.keys():                              # 단위 수정
        female_sum[shop] = sum(female[shop])/10000000000
    for shop in male.keys():
        male_sum[shop] = sum(male[shop])/10000000000

    # 그래프 그리기
    plt.figure(figsize=(8, 4))
    plt.rc('font', family='Malgun Gothic')
    plt.title('남녀 업종별 카드 이용금액 비교 그래프')
    plt.plot(male_sum.keys(),male_sum.values(), color='b', label='남성')
    plt.plot(female_sum.keys(),female_sum.values(), color='r', label='여성')
    plt.xlabel('업종')
    plt.ylabel('이용금액 (단위:100억)')
    plt.legend()
    plt.savefig('graph.png')
    plt.show()
    
    graph_img = PhotoImage(file='graph.png')
    graph_lbl.configure(image=graph_img)
    graph_lbl.image = graph_img

#--------------------------------------------------------   
# 하위목표 4: '식음료' 업종의 연령별 카드 이용금액 그래프
#--------------------------------------------------------    
def analysis4():
    sum_age = {}
    for row in data:
        if row[4] == '식음료':
            if row[6] == '20 미만' or row[6] == '20대미만':          # 통일되지 않은 연령 데이터 전처리
                age = '10대이하'
            else:
                age = row[6]
            if age not in sum_age.keys():                            # 연령별 이용금액 합산
                sum_age[age] = int(row[-1])
            else:
                sum_age[age] += int(row[-1])

    sum_age = dict(sorted(sum_age.items(), key=operator.itemgetter(0), reverse = True))

    age_li = list(sum_age.keys())

    for age in age_li:
        sum_age[age] = sum_age[age]/10000000000                      # 단위 수정

    # 그래프 그리기
    plt.figure(figsize=(8, 4))
    plt.rc('font', family='Malgun Gothic')
    plt.title('연령별 제주도 식음료 카드 이용 그래프')
    plt.xlabel('이용금액 (단위: 100억)')
    plt.barh(list(sum_age.keys()), list(sum_age.values()), color ='orange')
    plt.savefig('graph.png')
    plt.show()    
    
    graph_img = PhotoImage(file='graph.png')
    graph_lbl.configure(image=graph_img)
    graph_lbl.image = graph_img
    
#--------------------------------------------------------
# 추가 함수: 최고 이용금액이 어떤 달인지 구하기
#--------------------------------------------------------
def max_mon():
    sum_mon = [0]*12
    for row in data:
        if row[8][:4] == '2018':
            month = int(row[8][4:]) - 1
            sum_mon[month]+=int(row[-1])

    max_sum=max(sum_mon) 
    max_sum_mon=sum_mon.index(max_sum)+1                # 카드 이용금액이 가장 높은 달의 인덱스 값 확인

    if len(str(max_sum_mon)) != 2:
        max_sum_mon='0'+str(max_sum_mon)                # 이후 분석에서 활용할 수 있게 전처리 (예: 1 -> 01)
    return max_sum_mon
#--------------------------------------------------------
    
#################################################
# ----- 프로그램 화면 부분 -----
#################################################

root = Tk()
root.geometry('795x500')
root.title('📢당신은 JYP 팝업스토어에 오게 될 것입니다')

#-------------------------------
# ----- 프레임 -----
#-------------------------------

Graph = Frame(root)
Graph.grid(row=1, column=0)

#-------------------------------
# ----- 그래프 프레임 -----
#-------------------------------

btn1 = Button(Graph, text='1. 월별 이용금액', width=27, command=analysis1)
btn1.grid(row=0, column=0)

max_sum_mon = max_mon() 
btn2 = Button(Graph, text=f'2. {max_sum_mon}월 업종별 이용금액', width=27, command=analysis2)
btn2.grid(row=0, column=1)

btn3 = Button(Graph, text='3. 남녀 업종별 이용금액', width=27, command=analysis3)
btn3.grid(row=0, column=2)

btn4 = Button(Graph, text='4. 연령별 식음료 이용금액', width=27, command=analysis4)
btn4.grid(row=0, column=3)

graph_lbl = Label(Graph)
graph_lbl.grid(row=1, column=0, columnspan=5)

root.mainloop()